<a href="https://colab.research.google.com/github/jay-darvas/btcusd/blob/main/btcusd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas import read_csv
from numpy import zeros, array, sqrt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score, accuracy_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
scaler = MinMaxScaler()
df=read_csv('https://raw.githubusercontent.com/jay-darvas/btcusd/main/Bitfinex_BTCUSD_1h.csv', header=0, names=['Date', 'Price', 'Open',	'High',	'Low',	'Vol',	'Change']);
data=df.values
changes=[float(str(x).rstrip('\%')) for x in data[:,-1]]
price=[float(str(x).replace(',', '')) for x in data[:,1]]
vols=[float(str(x).rstrip('K')) for x in data[:,-2]]
N=len(changes)
X=[];y=[];
i=60
#i=40
#i=36
useDays=3200
#useDays=2400
#useDays=1200
#useDays=900
while i<N-useDays+1:
    ySum=sum(changes[i-60:i])#2 months ahead
    #ySum=sum(changes[i-40:i])#6 days ahead
    #ySum=sum(changes[i-36:i])#30 hours ahead
    #vol=sum(vols[i:i+useDays])
    #l=changes[i:i+useDays];
    l=price[i:i+useDays];
    v=vols[i:i+useDays];
    #l=l+v;
    X.insert(0,l)
    y.insert(0,ySum)
    i=i+1
X=array(X);y=array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=1)
X_train = scaler.fit_transform(X_train)
#We use the below X_test to predict the current day
#X_test=array(changes[0:useDays]+vols[0:useDays]).reshape(1,-1);
#X_test=array(price[0:useDays]+vols[0:useDays]).reshape(1,-1);
X_test=array(price[0:useDays]).reshape(1,-1);
X_test = scaler.transform(X_test)
#######################################################
print("X_train has the shape ", X_train.shape)
XGBModel = XGBRegressor()
#KNNModel= KNeighborsRegressor(n_neighbors=255, p=1, leaf_size=60, weights='distance', n_jobs=-1)
#KNNModel= KNeighborsRegressor(n_neighbors=255, weights='distance', n_jobs=-1)
cv = KFold(n_splits=2, shuffle=True, random_state=1)
#Here we list the parameters we want to tune
space = dict()
#search = GridSearchCV(KNNModel, space, n_jobs=-1, cv=cv, refit=True)
search = GridSearchCV(XGBModel, space, n_jobs=-1, cv=cv, refit=True)
result = search.fit(X_train, y_train)
best_model = result.best_estimator_
yhat_test = best_model.predict(X_test)
print("yhat_test is ",yhat_test);
yhat_train = best_model.predict(X_train)
print('RMSE train= ',sqrt(mean_squared_error(y_train, yhat_train)))
print('MAE train= ',mean_absolute_error(y_train, yhat_train))
print('R2 train:',r2_score(y_train, yhat_train))
print('RMSE test= ',sqrt(mean_squared_error(y_test, yhat_test)))
print('MAE test= ',mean_absolute_error(y_test, yhat_test))
print('R2 test:',r2_score(y_test, yhat_test))#original
print('Best parameters are',search.best_params_)#original
print('Hyperparameters: ',best_model.get_params())

X_train has the shape  (45714, 3200)
